## Second solution to assignment 2 with the use of ChatGPT
First, we start with the step by step solution.\
At the end, I will present the way to use the functionality without any effort with one function

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from openai import OpenAI
import os
import re
from tqdm import tqdm
import random
import string
import shutil
from typing import List
from datasets import load_dataset
import ast

For using ChatGPT, I use service Proxy API that provided additional endpoint by the link https://api.proxyapi.ru/openai/v1. It allows us to pay for tokens using rubles

In [8]:
proxy_api_key = "hehehehehehehehehehehehe"

client = OpenAI(
    api_key=proxy_api_key,
    base_url="https://api.proxyapi.ru/openai/v1",
)

In [9]:
# загрузка разных наборов данных

with open("../../data/train.jsonl", "r") as f:
    train_data = [json.loads(line) for line in f.readlines()]

with open("../../data/dev.jsonl", "r") as f:
    dev_data = [json.loads(line) for line in f.readlines()]

with open("../../data/test.jsonl", "r") as f:
    test_data = [json.loads(line) for line in f.readlines()]

Here we send every text from test set to ChatGPT with the task of finding all nested named entitities. The list of them is given in the context. The example of the solution is given too\
The output of the model is the string representation of list of lists, where each sublist is one entity of format ["text of the entity", "entity type"]

In [326]:
replies = [] 
for sample in tqdm(test_data):
    text = sample["senences"] # extracting text

    # asking for the entities
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=[{"role": "system", "content": "Пользователь даст текст. Тебе нужно найти все сущности в нем. Список сущностей на английском: AGE, AWARD, CITY, COUNTRY, CRIME, DATE, DISEASE, DISTRICT, EVENT, FACILITY, FAMILY, IDEOLOGY, LANGUAGE, LAW, LOCATION, MONEY, NATIONALITY, NUMBER, ORDINAL, ORGANIZATION, PENALTY, PERCENT, PERSON, PRODUCT, PROFESSION, RELIGION, STATE_OR_PROVINCE, TIME, WORK_OF_ART. Тебе нужно ответить без лишних комментариев в следующей форме: [[\"Газпром\", \"ORGANIZATION\"], [\"Роман\", \"PERSON\"]]. Например, для текста \"Я работаю учителем\", тебе нужно ответить [[\"учителем\", \"PROFESSION\"]], потому что в тексте есть упоминание профессии. Пиши как было в тексте, не меняй падеж и прочее!"},\
                                     {"role" : "system", "content" : "Еще пример выполнения задания. Часть правильного ответа к некоторому тексту: \"[[\"23-летнюю\", \"AGE\"], [\"Синьцзян-Уйгурского автономного района\", \"DISTRICT\"], [\"Ма Ай Лунь\", \"PERSON\"], [\"Китаянка\", \"NATIONALITY\"], [\"iPhone\", \"PRODUCT\"], [\"китаянки\", \"NATIONALITY\"], [\"убило током\", \"EVENT\"], [\"убило током\", \"EVENT\"], [\"погибла\", \"EVENT\"], [\"полиция\", \"ORGANIZATION\"], [\"гибель\", \"EVENT\"], [\"правоохранительные органы\", \"ORGANIZATION\"], [\"гибели\", \"EVENT\"]]\". Обрати внимание, что считается сущностью EVENT."},
                                     {"role": "user", "content": text}\
                                        ])
    
    # extracting the text answer out of answer object
    reply = chat_completion.choices[0].message.content
    replies.append(reply)

100%|██████████| 65/65 [04:10<00:00,  3.85s/it]


In [334]:
# Here is the example of extracted entities for the first text
replies[0]

'[["Бирмингем Сити", "ORGANIZATION"], ["Карсон Ёнг", "PERSON"], ["Гонконга", "CITY"], ["55 миллионов фунтов стерлингов", "MONEY"], ["2001", "DATE"], ["2007 годы", "DATE"], ["Карсон Ёнг", "PERSON"], ["Бирмингема", "ORGANIZATION"], ["2009 году", "DATE"], ["81,5 миллионов фунтов стерлингов", "MONEY"]]'

In [57]:
ids = [s["id"] for s in test_data]

The last thing to do is to transform the output to the desired format the system wants and to save the answer

In [330]:
with open("test.jsonl", "w") as f:
    for i, reply in enumerate(replies):

        # try except for debugging purposes
        try:
            # The output of the model is string with the list we need. This function transforms the string of list of lists TO list of lists
            list_of_entities = ast.literal_eval(reply.replace("\"\"", "\""))
        except:
            print(i)
            break
        
        text = test_data[i]["senences"].lower()
        entities_in_good_format = [] # This is the list of entities in the desired format [1,5, "PERSON"] instead of ["Roman", "PERSON"]
        # in this loop we replace entities by their locations in the text
        for entity in list_of_entities:
            word = entity[0]
            entity_type = entity[1]

            start = text.find(word.lower())
            if start == -1: # did not found such entity. happens sometimes, just skip
                continue
            end = start + len(word.lower()) - 1

            entities_in_good_format.append([start, end, entity_type])
            
        id = ids[i]
        d = {"id" : id, "ners" : entities_in_good_format}
        f.write(str(d).replace("'", "\""))
        f.write("\n")

In [333]:
# .jsonl to .zip
!zip test test.jsonl

updating: test.jsonl (164 bytes security) (deflated 77%)


In [7]:
shutil.move("./test.jsonl", "./output/test.jsonl")

'./output/test.jsonl'

In [8]:
shutil.move("./test.zip", "./output/test.zip")

'./output/test.zip'

### Example of usage for any text easily

Above was the step by step solution like the stream of code. Here is the example how to apply the same functionality on your text using the predefined function you can find in the file NNER_model\
It requires only the text to process and the token for the Proxy API

In [2]:
from NNER_model import extract_entities

In [3]:
extract_entities("Hello. My name is Roman", proxy_api_key)

[[18, 22, 'PERSON']]